# Assignment 1. Artificial Intelligence and Intelligent Agents #

<img src="./ExplorerKids.jpg" width="400">

# The Explorer #

Our agent's name is Explorer, first we are exporting the aima's class known as agents.

In [1]:
from agents import *

## Things
Now we are defining all the "things" that are goint to be place inside the environment.

We define Treasure as a subclass of the superclass thing and then Gold and Diamond as a subclass of Treasure. 

Also for the Traps we define the subclass Trap first and then the Hole and Snake as kinds of traps.

In [2]:
class Treasure(Thing):
    def __init__(self):
        self.value = 10
        
class Diamond(Treasure):
    def __init__(self):
        self.value = 20
        
class Gold(Treasure):
    def __init__(self):
        self.value = 10
        
class Trap(Thing):
    def __init__(self):
        self.value = 10
        
class Snakes(Trap):
    def __init__(self):
        self.value = 20
        
class Hole(Trap):
    def __init__(self):
        self.value = 10 
        
class Wall(Thing):
        pass
        
class Empty(Thing):
        pass
        
class Out(Thing):
    pass

## The Agent (Explorer)

<img src="./explor.jpg" width="400">

Now we define the Agent called Explorer and all its attributes, 

An internal model with a grid 6x6 full of question marks "?".

The movement options (move_right, move_left, move_up, move_down).

The way it relates with the things inside the environment, to be able to get a treasure (gold or diamond) (get_treasure) that will add to the performance, get inside a trap (get_trapped) that will substract to the performance, hit a wall (bump) or get out the environment once it is done (noOp).

The percepts that it can get (add_percepts), and the internal model of the agent (draw_model).

In [12]:
class Explorer(Agent):
    def __init__(self, program=None):
        super().__init__()
        self.model = {}
        self.isTrapped = False
        self.program = program
        
        for i in range(0, 6):
            for j in range(0,6):
                self.model[j,i] = '?'
                
    def move_right(self):
        self.location = (self.location[0]+1, self.location[1])
        self.performance -= 1
        self.isTrapped = False
        
    def move_left(self):
        self.location = (self.location[0]-1, self.location[1])
        self.performance -= 1
        self.isTrapped = False
        
    def move_up(self):
        self.location = (self.location[0], self.location[1]-1)
        self.performance -= 1
        self.isTrapped = False
    
    def move_down(self):
        self.location = (self.location[0], self.location[1]+1)
        self.performance -= 1
        self.isTrapped = False
        
    def get_treasure(self, thing):
        if isinstance(thing, Treasure):
            print("Explorer: found Treasure at {}.".format(self.location))
            self.performance += thing.value
            return True
        return False
        
    def get_trapped(self, thing):
        if isinstance(thing, Trap):
            print("Explorer: fell in a Trap at {}.".format(self.location))
            self.performance -= thing.value
            self.isTrapped = True
            return True
        return False
        
    def bump(self, thing):
        if isinstance(thing, Wall):
            print("Explorer: bumped against a wall at {}.".format(self.location))
            return True
        return False
        
    def noOp(self):
        print("Explorer finished its job")
        return
        
        
    def add_percepts(self, percepts):
        thing_percepts = {
            Diamond: 'd',
            Gold: 'g',
            Wall: 'w',
            Empty: '-',
            Snakes: 's',
            Hole: 'h',
            Explorer: 'a'}
        for p in percepts:
            if (len(p)==1):
                obj = [thing_percepts.get(t.__class__, t) for t in p]
                self.model[p[0].location]=obj[0]
                
            if (len(p)==2):
                obj = [thing_percepts.get(t.__class__, t) for t in p 
                if not isinstance(t, Agent)]
                self.model[p[0].location]=obj[0]
                
                
                
        
        
    def draw_model(self):
        for i in range (0,6):
            for j in range (0,6):
                
                x = self.model[(j, i)]
                
                if((j, i)==self.location):
                    print('a', end="")
                else:
                    print(x, end="")
                                
            print()
        print()
        

## The Grid (Environment)

Then we define the environment, as follows:

* The percept, composed by the four locations around the agent (left, right, down, up) and the position the agent is standing in.

&nbsp;

* The things that we are going to add to the map to conform the environment.

&nbsp;

* The process to create the grid with a random function that will add the things with different probabilities.

&nbsp;

* The actions that the agent will be able to do inside the grid.

&nbsp;

* Finally the step, that is how the agent will move inside the environment.

In [13]:
class grid(Environment):
        
    
    def percept(self, agent):
        '''Returns things in adjacent (not diagonal) cells of the agent.
            Result format: [Left, Right, Up, Down, Center / Current location]'''
        
        x, y = agent.location
        result = []
        result.append(self.list_things_at((x, y)))
        result.append(self.list_things_at((x - 1, y)))
        result.append(self.list_things_at((x + 1, y)))
        result.append(self.list_things_at((x, y - 1)))
        result.append(self.list_things_at((x, y + 1)))
        
        return result
        
    def insert_thing(self, thing, location):
        items = self.list_things_at(location, tclass=Thing)
        for item in items:
            if(not isinstance(item, Agent)):
                self.delete_thing(item)
                self.add_thing(thing, location)
    
    def create_grid_rand(self, size):
        self.size=size
        for i in range (0,self.size):
            for j in range (0,self.size):
                x = random.randint(0,99)
                if x <= 60: 
                    self.add_thing(Empty(), (j,i))
                elif x<=70:
                    self.add_thing(Wall(), (j,i))
                elif x<=80: 
                    self.add_thing(Gold(), (j,i))
                elif x<=90: 
                    self.add_thing(Hole(), (j,i))
                elif x<=95: 
                    self.add_thing(Diamond(), (j,i))
                else: 
                    self.add_thing(Snakes(), (j,i))
                    
        items = self.list_things_at((0,0), tclass=Thing)
        for item in items:
            if(not isinstance(item, Agent) and not isinstance(item, Empty)):
                self.delete_thing(item)
                self.add_thing(Empty(), (0,0))
                
        return True
            
    def draw_grid(self):
        thing_percepts = {
            Diamond: 'd',
            Gold: 'g',
            Wall: 'w',
            Empty: '-',
            Snakes: 's',
            Hole: 'h',
            Explorer: 'a'}
           
        for i in range (0,6):
            for j in range (0,6):
                
                x = self.list_things_at((j, i))
                
                obj = [thing_percepts.get(t.__class__, t) for t in x]
                if(len(obj)==2):
                    obj = ['a'] 
                print(obj[0], end="")
                                
            print()
        print()
            
    def execute_action(self, agent, action):
        "Change the world to reflect this action."
        if action == "right":
            agent.move_right()
        elif action == "left":
            agent.move_left()
        elif action == "up":
            agent.move_up()
        elif action == "down":
            agent.move_down()
        elif action == "get treasure":
            items = self.list_things_at(agent.location, tclass=Treasure)
            if len(items) != 0:
                agent.get_treasure(items[0])
                self.delete_thing(items[0])
                self.add_thing(Empty(), agent.location)
        elif action == "get trapped":
            items = self.list_things_at(agent.location, tclass=Trap)
            if len(items) != 0 and not agent.isTrapped:
                agent.get_trapped(items[0])
        
        elif action == "noOp":
            agent.noOp()
        
        print("Performance: ", agent.performance)
                   
        return 
        
    def step(self):
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    a_percepts = self.percept(agent)
                    agent.add_percepts(a_percepts)
                    actions.append(agent.program(a_percepts))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()
        self.draw_grid()
        agent.draw_model()


## Program

Now we save the percepts to the model of the agent, that way it will be able to explore the environment and store the new information.

The agent will look inside its own model and look for not explored locations "?", diamonds "d" or gold "g", if it does not find any, it will execute noOp instruction and be finished with the exploration. 

If the agent sees a treasure around then it will go and get it by means of the "get_treasure" instruccion, if it does decide to step into a trap it will follow the "get_trapped" instruccion.

In [22]:
def programAgent(percept):
    ''' Recibe percepts de lo que hay a su alrededor (posicion actual, izq, der, arriba, abajo) '''
    
    list_things = [Diamond, Gold, Empty, Hole, Snakes, Wall]
        
    for p in percept[0]:
        if isinstance(p, Agent):
            ag = p
    
    l_model = [ag.model[i] for i in ag.model]
    if(('?' not in l_model)and('g' not in l_model)and('d' not in l_model)):
        return "noOp"
            
    
    for p in percept[0]:
        if isinstance(p, Treasure):
            return 'get treasure'
        elif isinstance(p, Trap):
            return 'get trapped'
            
    for thing in list_things:
        t = [] # Treasures matrix
        for p1 in percept:
            if(len(p1)>0):
                if isinstance(p1[0], thing):
                    t.append(p1[0])
        if(len(t)>0):
            rn = random.randint(0, len(t)-1)
            move = (ag.location[0] - t[rn].location[0], ag.location[1] - t[rn].location[1])
            if(move[0]==1):
                return "left"
            elif(move[0]==-1):
                return "right"
            elif(move[1]==1):
                return "up"
            elif(move[1]==-1):
                return "down"
            
    return 'right'        

## Running the program

<img src="./scared-exp.jpg" width="400">

We initialize a variable g by calling the class grid(), then the Explorer (a).

After wrapping the agent to follow its movements.

we procedd to Add the agent to the grid in the position (0,0).

Create a random grid of 6x6.

And printing the grid with the percepts that the Explorer has found.

And we decide to run for as many steps as we want, if the Explorer is donde before the total steps we stablished, then it 
will stop moving and that will prevent from substracting more in the Agent's performance.

In [23]:
g = grid()

a=Explorer(program = programAgent)
TraceAgent(a)
g.add_thing(a, (0,0))
g.create_grid_rand(6)
g.draw_grid()

g.percept(a)
g.run(200)

Can't find a valid program for Explorer, falling back to default.
a----g
-g-ddg
-s-gh-
-d--g-
--wh-d
w-----

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Empty>], [], [<Empty>]] and does down
Performance:  -1
-----g
ag-ddg
-s-gh-
-d--g-
--wh-d
w-----

--????
a?????
??????
??????
??????
??????

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Gold>], [<Empty>], [<Empty>]] and does right
Performance:  -2
-----g
-a-ddg
-s-gh-
-d--g-
--wh-d
w-----

--????
-a????
-?????
??????
??????
??????

<Explorer> perceives [[<Explorer>, <Gold>], [<Empty>], [<Empty>], [<Empty>], [<Snakes>]] and does get treasure
Explorer: found Treasure at (1, 1).
Performance:  8
-----g
-a-ddg
-s-gh-
-d--g-
--wh-d
w-----

--????
-a-???
-s????
??????
??????
??????

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], [<Snakes>]] and does up
Performance:  7
-a---g
---ddg
-s-gh-
-d--g-
--wh-d
w-----

-a????
---???
-s????
??????
??????
??????

<Explorer> perceives [[<Explorer>, <Empty>], [<

------
------
-s--h-
---a--
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], [<Hole>]] and does right
Performance:  71
------
------
-s--h-
----a-
--wh-d
w-----

------
------
-s--h-
----a-
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Hole>], [<Empty>]] and does left
Performance:  70
------
------
-s--h-
---a--
--wh-d
w-----

------
------
-s--h-
---a--
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], [<Hole>]] and does left
Performance:  69
------
------
-s--h-
--a---
--wh-d
w-----

------
------
-s--h-
--a---
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], [<Wall>]] and does up
Performance:  68
------
------
-sa-h-
------
--wh-d
w-----

------
------
-sa-h-
------
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [<Snakes>], [<Empty>], [<Empty>], [<Empty>]] and does down
Performance:  67
------
------
-s-

------
--wh-d
w-----

------
a-----
-s--h-
------
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Empty>], [<Empty>], [<Empty>]] and does down
Performance:  22
------
------
as--h-
------
--wh-d
w-----

------
------
as--h-
------
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Snakes>], [<Empty>], [<Empty>]] and does up
Performance:  21
------
a-----
-s--h-
------
--wh-d
w-----

------
a-----
-s--h-
------
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Empty>], [<Empty>], [<Empty>]] and does down
Performance:  20
------
------
as--h-
------
--wh-d
w-----

------
------
as--h-
------
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Snakes>], [<Empty>], [<Empty>]] and does up
Performance:  19
------
a-----
-s--h-
------
--wh-d
w-----

------
a-----
-s--h-
------
--wh-?
w--???

<Explorer> perceives [[<Explorer>, <Empty>], [], [<Empty>], [<Empty>], [<Empty>]] and does up
Performance:  18
a-----
------
-s--h-
------
--

Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<E

-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer>, <Empty>], [<Empty>], [<Empty>], [<Empty>], []] and does noOp
Explorer finished its job
Performance:  1
------
------
-s--h-
------
--wh--
w---a-

------
------
-s--h-
------
--wh--
w---a-

<Explorer> perceives [[<Explorer